# BTO Cost Estimator Agent Data Cleaning

# Data Preparation
Parse BTO text copied from **HDB Annexes** on **BTO Flat Supply and Pricing Details**

In [21]:
import pandas as pd
import re
bto_df = pd.DataFrame()

In [22]:
def parse_bto_text(table_text):
    lines = [line.strip() for line in table_text.strip().split('\n') if line.strip()]
    data = []

    current_exercise = None
    current_project_type = None
    current_project = None
    current_wait_time = None

    for line in lines:
        # Detect exercise name (e.g., "Feb 2025")
        if re.match(r'^[A-Za-z]{3,9}\s+\d{4}$', line):
            current_exercise = line
            continue

        # Detect Project Type
        if line in ["Standard Projects", "Plus Project", "Prime Project"]:
            current_project_type = line
            continue

        # Detect Project + Waiting Time
        if "$" not in line and re.search(r'\d+(/?\d*)$', line):
            parts = line.rsplit(" ", 1)
            if len(parts) == 2:
                current_project = parts[0]
                current_wait_time = parts[1]
            else:
                current_project = line
            continue

        # Detect flat row (contains price)
        if "$" in line:
            # Extract price
            price_match = re.search(r'\$(\d{1,3}(?:,\d{3})*)\s*-\s*\$(\d{1,3}(?:,\d{3})*)', line)
            if not price_match:
                continue
            min_price = int(price_match.group(1).replace(",", ""))
            max_price = int(price_match.group(2).replace(",", ""))

            # Remove price part
            row_text = line[:price_match.start()].strip()
            tokens = row_text.split()

            # Detect flat type
            if len(tokens) > 1 and tokens[1].lower() == "flexi":
                flat_type = " ".join(tokens[:2])
                tokens = tokens[2:]
            else:
                flat_type = tokens[0]
                tokens = tokens[1:]

            # Start from the right to parse units & internal floor
            units = tokens[-1]

            # Check if internal floor is a range
            if "-" in tokens[-3:-1]:
                est_internal_area = " ".join(tokens[-4:-1])
                floor_tokens_end = -4
            else:
                est_internal_area = tokens[-2]
                floor_tokens_end = -2

            # Remaining tokens = floor area
            est_floor_area = " ".join(tokens[:floor_tokens_end])

            # Normalize ranges
            est_floor_area = re.sub(r"\s*-\s*", " - ", est_floor_area)
            est_internal_area = re.sub(r"\s*-\s*", " - ", est_internal_area)

            data.append({
                "Exercise": current_exercise,
                "Project Type": current_project_type,
                "Project": current_project,
                "Waiting Time": current_wait_time,
                "Flat Type": flat_type,
                "Estimated Floor Area": est_floor_area,
                "Estimated Internal Floor Area": est_internal_area,
                "Units": int(units),
                "Min Price": min_price,
                "Max Price": max_price
            })

    return pd.DataFrame(data)

In [23]:
table_text = """
Oct 2023
Standard Projects
Rail Green I & II @ CCK 39/48
2-room Flexi 38 36 114 $106,000 - $132,000
2-room Flexi 48 46 283 $134,000 - $168,000
3-room 69 66 152 $216,000 - $272,000
4-room 93 - 95 90 - 92 802 $319,000 - $437,000
5-room 113 - 115 110 - 112 518 $463,000 - $595,000
3Gen 120 115 26 $471,000 - $530,000
Plantation Edge I & II 36/40
2-room Flexi 38 36 52 $114,000 - $133,000
2-room Flexi 48 46 211 $142,000 - $183,000
3-room 69 66 81 $232,000 - $279,000
4-room 93 90 353 $353,000 - $418,000
5-room 113 110 313 $460,000 - $568,000
Plus Project
Rajah Residences 52
2-room Flexi 38 36 31 $176,000 - $216,000
2-room Flexi 48 46 101 $214,000 - $294,000
4-room 93 90 607 $480,000 - $675,000
Tenteram Vantage 45
3-room 69 66 88 $339,000 - $420,000
4-room 93 90 952 $488,000 - $628,000
Prime Project
Verandah @ Kallang 42
2-room Flexi 39 36 28 $193,000 - $237,000
2-room Flexi 49 46 112 $240,000 - $303,000
3-room 69 66 110 $368,000 - $475,000
4-room 93 90 893 $535,000 - $675,000
Tanglin Halt Cascadia 54
3-room 66 63 155 $364,000 - $509,000
4-room 89 86 818 $537,000 - $702,000
"""

df = parse_bto_text(table_text)
df


,Exercise,Project Type,Project,Waiting Time,Flat Type,Estimated Floor Area,Estimated Internal Floor Area,Units,Min Price,Max Price
0,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,2-room Flexi,38,36,114,106000,132000
1,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,2-room Flexi,48,46,283,134000,168000
2,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,3-room,69,66,152,216000,272000
3,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,4-room,93 - 95,90 - 92,802,319000,437000
4,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,5-room,113 - 115,110 - 112,518,463000,595000
5,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,3Gen,120,115,26,471000,530000
6,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,2-room Flexi,38,36,52,114000,133000
7,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,2-room Flexi,48,46,211,142000,183000
8,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,3-room,69,66,81,232000,279000
9,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,4-room,93,90,353,353000,418000


In [24]:
bto_df = pd.concat([bto_df, df], ignore_index=True)
bto_df

,Exercise,Project Type,Project,Waiting Time,Flat Type,Estimated Floor Area,Estimated Internal Floor Area,Units,Min Price,Max Price
0,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,2-room Flexi,38,36,114,106000,132000
1,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,2-room Flexi,48,46,283,134000,168000
2,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,3-room,69,66,152,216000,272000
3,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,4-room,93 - 95,90 - 92,802,319000,437000
4,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,5-room,113 - 115,110 - 112,518,463000,595000
5,Oct 2023,Standard Projects,Rail Green I & II @ CCK,39/48,3Gen,120,115,26,471000,530000
6,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,2-room Flexi,38,36,52,114000,133000
7,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,2-room Flexi,48,46,211,142000,183000
8,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,3-room,69,66,81,232000,279000
9,Oct 2023,Standard Projects,Plantation Edge I & II,36/40,4-room,93,90,353,353000,418000


In [ ]:
# Export to CSV
# bto_df = pd.read_csv("bto_pricing_detail.csv")
# bto_df.to_csv("bto_pricing_detail.csv", index=False)

# Data Cleaning

In [10]:
import pandas as pd
import numpy as np
import re

def parse_range(value):
    """parse a numeric range like '93 - 96' and return the average as float"""
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float)):
        return float(value)
    # match "min - max" or "min–max" (handle dash variants)
    match = re.match(r'(\d+(?:\.\d+)?)\s*[-–]\s*(\d+(?:\.\d+)?)', str(value))
    if match:
        low, high = match.groups()
        return (float(low) + float(high)) / 2
    # if single number
    try:
        return float(value)
    except ValueError:
        return np.nan

def clean_bto_data(csv_path: str) -> pd.DataFrame:
    """load and clean bto pricing"""
    df = pd.read_csv(csv_path)

    # strip column names and lower-case
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # normalize key text columns
    for col in ["project", "flat_type", "project_type"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()

    # parse exercise as datetime (assume format like "Feb 2025")
    if "exercise" in df.columns:
        df["exercise_date"] = pd.to_datetime(df["exercise"], format="%b %Y", errors="coerce")

    # parse numeric ranges in floor area columns
    for col in ["estimated_floor_area", "estimated_internal_floor_area"]:
        if col in df.columns:
            df[col] = df[col].apply(parse_range)

    # ensure numeric columns
    for col in ["units", "min_price", "max_price"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # compute average price
    if "min_price" in df.columns and "max_price" in df.columns:
        df["avg_price"] = df[["min_price", "max_price"]].mean(axis=1)

    # optional: compute average floor area (external + internal)
    if "estimated_floor_area" in df.columns and "estimated_internal_floor_area" in df.columns:
        df["avg_floor_area"] = df[["estimated_floor_area", "estimated_internal_floor_area"]].mean(axis=1)

    # drop rows with no useful price data
    df = df.dropna(subset=["avg_price"])

    return df


In [19]:
df_clean = clean_bto_data("bto_pricing_detail.csv")
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   exercise                       201 non-null    object        
 1   project_type                   201 non-null    object        
 2   project                        201 non-null    object        
 3   waiting_time                   201 non-null    object        
 4   flat_type                      201 non-null    object        
 5   estimated_floor_area           201 non-null    float64       
 6   estimated_internal_floor_area  201 non-null    float64       
 7   units                          201 non-null    int64         
 8   min_price                      201 non-null    int64         
 9   max_price                      201 non-null    int64         
 10  exercise_date                  201 non-null    datetime64[ns]
 11  avg_price          

In [31]:
# save to new csv
df_clean.to_csv("bto_pricing_detail_cleaned.csv", index=False)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   exercise                       201 non-null    object        
 1   project_type                   201 non-null    object        
 2   project                        201 non-null    object        
 3   waiting_time                   201 non-null    object        
 4   flat_type                      201 non-null    object        
 5   estimated_floor_area           201 non-null    float64       
 6   estimated_internal_floor_area  201 non-null    float64       
 7   units                          201 non-null    int64         
 8   min_price                      201 non-null    int64         
 9   max_price                      201 non-null    int64         
 10  exercise_date                  201 non-null    datetime64[ns]
 11  avg_price          